# 一、相关包准备

In [1]:
# 导入包
import jieba
import pandas as pd
import re
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

# 二、加载数据并进行数据预处理

In [2]:
# 加载数据
df = pd.read_excel(r"C:\Users\24756\Desktop\filtered_cnews.train.xlsx")
raw_texts = df['内容'].tolist()
labels = df['类别'].tolist()

# 打印总样本数
print(f"总样本数: {len(raw_texts)}")
# 打印前5个样本验证数据
print("\n原始文本示例：")
for i in range(5):
    print(f"文本{i+1}: {raw_texts[i][:30]}...")  # 只显示前30个字符
# 打印类别及其数量
print("\n类别分布:")
print(pd.Series(labels).value_counts())

总样本数: 30000

原始文本示例：
文本1: 	马晓旭意外受伤让国奥警惕 无奈大雨格外青睐殷家军记者傅亚雨...
文本2: 	商瑞华首战复仇心切 中国玫瑰要用美国方式攻克瑞典多曼来了，...
文本3: 	冠军球队迎新欢乐派对 黄旭获大奖张军赢下PK赛新浪体育讯1...
文本4: 	辽足签约危机引注册难关 高层威逼利诱合同笑里藏刀新浪体育讯...
文本5: 	揭秘谢亚龙被带走：总局电话骗局 复制南杨轨迹体坛周报特约记...

类别分布:
家居    5000
教育    5000
财经    5000
科技    5000
房产    5000
体育    5000
dtype: int64


In [3]:
# 加载停用词
stopwords = open(r'C:\Users\24756\Desktop\stopwords.txt','r',encoding="utf-8").read()
stopwords = stopwords.split('\n')
# 展示前15个停用词
print("停用词: ",",".join(stopwords[:15]))
stopwords = set(stopwords) # 转为集合，便于搜索

processed_texts = []
# 文本预处理
for text in raw_texts:
    # 首先，将非中文的内容使用空格代替，并且合并多个空格为一个
    text = re.sub(r"[^\u4e00-\u9fa5]", " ", text)
    text = re.sub(r"\s+", " ", text)
    # jieba分词
    words = jieba.cut(text)
    # 去除停用词
    filtered_words = [word for word in words if word not in stopwords]
    processed_texts.append(filtered_words)
# 打印前2个处理后的文本
print(processed_texts[:2])

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\24756\AppData\Local\Temp\jieba.cache


停用词:  <PAD>,，,的,。,一,是,在,0,有,不,了,中,1,人,大


Loading model cost 0.724 seconds.
Prefix dict has been built successfully.


[['马晓旭', '意外', '受伤', '国奥', '警惕', '无奈', '大雨', '格外', '青睐', '殷家', '记者', '傅亚雨', '沈阳', '报道', '来到', '沈阳', '国奥队', '依然', '没有', '摆脱', '雨水', '困扰', '下午', '国奥队', '日常', '训练', '再度', '受到', '大雨', '干扰', '无奈', '之下', '队员', '慢跑', '分钟', '草草收场', '上午', '国奥队', '奥体中心', '外场', '训练', '时候', '就是', '阴沉沉', '气象预报', '显示', '当天', '下午', '沈阳', '大雨', '幸好', '队伍', '上午', '训练', '没有', '受到', '任何', '干扰', '下午', '球队', '抵达', '训练场', '大雨', '已经', '几个', '小时', '而且', '丝毫', '没有', '停下来', '意思', '试一试', '态度', '球队', '开始', '当天', '下午', '例行', '训练', '分钟', '过去', '天气', '没有', '任何', '转好', '迹象', '为了', '保护', '球员', '国奥队', '决定', '中止', '当天', '训练', '全队', '立即', '返回', '酒店', '训练', '足球队', '来说', '不是', '什么', '稀罕', '奥运会', '即将', '开始', '之前', '全队', '变得', '娇贵', '沈阳', '最后', '一周', '训练', '国奥队', '首先', '保证', '现有', '球员', '不再', '出现意外', '伤病', '情况', '以免', '影响', '正式', '比赛', '因此', '这一', '阶段', '控制', '训练', '受伤', '控制', '感冒', '疾病', '出现', '队伍', '放在', '相当', '重要', '位置', '抵达', '沈阳', '之后', '后卫', '冯萧霆', '一直', '没有', '训练', '冯萧霆', '长春', '患上', '感冒', '因此', '没有', '参加', '塞尔维亚', '热身赛', '队伍', '介绍', 

# 三、划分数据集

In [4]:
# 类别均衡划分数据集
X_train1, X_temp, y_train1, y_temp = train_test_split(processed_texts, labels, test_size = 0.2, stratify = labels, random_state = 42)
X_val1, X_test1, y_val1, y_test1 = train_test_split(X_temp, y_temp, test_size = 0.5, stratify = y_temp, random_state = 42)

# 打印各数据集大小
print(f"训练集大小: {len(X_train1)}")
print(f"验证集大小: {len(X_val1)}")
print(f"测试集大小: {len(X_test1)}")

# 打印每个类别的分布
print("\n训练集类别分布:\n")
print(pd.Series(y_train1).value_counts())

print("\n验证集类别分布:")
print(pd.Series(y_val1).value_counts())

print("\n测试集类别分布:")
print(pd.Series(y_test1).value_counts())

训练集大小: 24000
验证集大小: 3000
测试集大小: 3000

训练集类别分布:

家居    4000
教育    4000
财经    4000
科技    4000
房产    4000
体育    4000
dtype: int64

验证集类别分布:
财经    500
科技    500
房产    500
体育    500
教育    500
家居    500
dtype: int64

测试集类别分布:
财经    500
科技    500
房产    500
体育    500
教育    500
家居    500
dtype: int64


In [5]:
# 随机划分数据集（类别不均衡）
X_train2, X_temp, y_train2, y_temp = train_test_split(processed_texts, labels, test_size=0.2, random_state=42)
X_val2, X_test2, y_val2, y_test2 = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# 打印各数据集大小
print(f"训练集大小: {len(X_train2)}")
print(f"验证集大小: {len(X_val2)}")
print(f"测试集大小: {len(X_test2)}")

# 打印每个类别的分布
print("\n训练集类别分布:")
print(pd.Series(y_train2).value_counts())

print("\n验证集类别分布:")
print(pd.Series(y_val2).value_counts())

print("\n测试集类别分布:")
print(pd.Series(y_test2).value_counts())

训练集大小: 24000
验证集大小: 3000
测试集大小: 3000

训练集类别分布:
教育    4050
财经    4031
体育    4022
家居    3976
科技    3967
房产    3954
dtype: int64

验证集类别分布:
科技    529
房产    513
家居    509
体育    487
教育    486
财经    476
dtype: int64

测试集类别分布:
房产    533
家居    515
科技    504
财经    493
体育    491
教育    464
dtype: int64


# 四、提取特征与特征选择

In [6]:
# 文本向量化
tfidf1 = TfidfVectorizer(tokenizer=lambda x: x, lowercase=False)
tfidf2 = TfidfVectorizer(tokenizer=lambda x: x, lowercase=False)

# 均衡数据集
X_train_tfidf1 = tfidf1.fit_transform(X_train1)
X_val_tfidf1 = tfidf1.transform(X_val1)
X_test_tfidf1 = tfidf1.transform(X_test1)

# 不均衡数据集
X_train_tfidf2 = tfidf2.fit_transform(X_train2)
X_val_tfidf2 = tfidf2.transform(X_val2)
X_test_tfidf2 = tfidf2.transform(X_test2)

In [7]:
# 利用卡方统计量进行特征选择
# 选择最佳3000个特征
k = 3000

# 对均衡数据集应用卡方检验
selector1 = SelectKBest(chi2, k = k)
X_train_chi1 = selector1.fit_transform(X_train_tfidf1, y_train1)
X_val_chi1 = selector1.transform(X_val_tfidf1)
X_test_chi1 = selector1.transform(X_test_tfidf1)

# 对不均衡数据集应用卡方检验
selector2 = SelectKBest(chi2, k = k)
X_train_chi2 = selector2.fit_transform(X_train_tfidf2, y_train2)
X_val_chi2 = selector2.transform(X_val_tfidf2)
X_test_chi2 = selector2.transform(X_test_tfidf2)

# 五、模型训练与评估

In [8]:
# 定义一个函数评估模型性能
classes = sorted(pd.Series(labels).unique())
print("排序后的类别顺序:", classes)
def evaluate_model(y_true, y_pred):
    # 报告混淆矩阵（转化为数值矩阵）
    cm = confusion_matrix(y_true, y_pred,labels = classes)
    # 转换为 DataFrame 并添加行列标签
    cm_df = pd.DataFrame(
        cm,
        index = pd.Index(classes, name = "真实类别"),  # 行标签为真实类别
        columns = pd.Index(classes, name = "预测类别")  # 列标签为预测类别
    )
    print("Confusion Matrix:")
    print(cm_df.to_string())  # 避免省略行列
    
    # 报告精确度、召回率、F1分数、支持度
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred))
    
    # 计算并报告宏平均与微平均F1分数，宏平均更适用于类别均衡，微平均更适用于类别不均衡
    macro_f1 = f1_score(y_true, y_pred, average='macro')
    micro_f1 = f1_score(y_true, y_pred, average='micro')
    print(f"Macro F1 Score: {macro_f1:.5f}")
    print(f"Micro F1 Score: {micro_f1:.5f}")

排序后的类别顺序: ['体育', '家居', '房产', '教育', '科技', '财经']


In [9]:
# 不同模型不同数据集进行文本分类，观察分类结果
models = {
    "Naive Bayes": MultinomialNB(),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "GBDT": GradientBoostingClassifier(n_estimators=50, random_state=42)
}

for name, model in models.items():
    # 处理均衡数据集
    model.fit(X_train_chi1, y_train1)
    y_pred_val1 = model.predict(X_val_chi1)
    print(f"\n{name}（类别均衡验证集）:")
    evaluate_model(y_val1, y_pred_val1)
    
    y_pred_test1 = model.predict(X_test_chi1)
    print(f"\n{name}（类别均衡测试集）:")
    evaluate_model(y_test1, y_pred_test1)
    print("\n" + "-"*60 + "\n")
    
    # 处理不均衡数据集
    model.fit(X_train_chi2, y_train2)
    y_pred_val2 = model.predict(X_val_chi2)
    print(f"\n{name}（类别不均衡验证集）:")
    evaluate_model(y_val2, y_pred_val2)
   
    y_pred_test2 = model.predict(X_test_chi2)
    print(f"\n{name}（类别不均衡测试集）:")
    evaluate_model(y_test2, y_pred_test2)
    print("\n" + "="*100 + "\n")


Naive Bayes（类别均衡验证集）:
Confusion Matrix:
预测类别   体育   家居   房产   教育   科技   财经
真实类别                              
体育    499    0    0    1    0    0
家居      3  471   15    4    6    1
房产      0    4  478    4    1   13
教育      1    5    3  471   19    1
科技      0    4    2    7  484    3
财经      1    6   15   11    9  458

Classification Report:
              precision    recall  f1-score   support

          体育       0.99      1.00      0.99       500
          家居       0.96      0.94      0.95       500
          房产       0.93      0.96      0.94       500
          教育       0.95      0.94      0.94       500
          科技       0.93      0.97      0.95       500
          财经       0.96      0.92      0.94       500

    accuracy                           0.95      3000
   macro avg       0.95      0.95      0.95      3000
weighted avg       0.95      0.95      0.95      3000

Macro F1 Score: 0.95361
Micro F1 Score: 0.95367

Naive Bayes（类别均衡测试集）:
Confusion Matrix:
预测类别   体育   家居   房产   教


GBDT（类别不均衡验证集）:
Confusion Matrix:
预测类别   体育   家居   房产   教育   科技   财经
真实类别                              
体育    485    1    0    0    1    0
家居      0  499    0    3    3    4
房产      0    3  507    0    0    3
教育      2   29    2  435   10    8
科技      2   24    2   11  483    7
财经      0   14   10    8    6  438

Classification Report:
              precision    recall  f1-score   support

          体育       0.99      1.00      0.99       487
          家居       0.88      0.98      0.92       509
          房产       0.97      0.99      0.98       513
          教育       0.95      0.90      0.92       486
          科技       0.96      0.91      0.94       529
          财经       0.95      0.92      0.94       476

    accuracy                           0.95      3000
   macro avg       0.95      0.95      0.95      3000
weighted avg       0.95      0.95      0.95      3000

Macro F1 Score: 0.94900
Micro F1 Score: 0.94900

GBDT（类别不均衡测试集）:
Confusion Matrix:
预测类别   体育   家居   房产   教育   科技   财经


1、总得来看，测试集结果与验证集相差不大，分类准确率较好的模型是GBDT。  
2、此外，值得注意的是，朴素贝叶斯、GBDT模型在进行特征选择和不进行特征选择的时候准确率差不多；但是KNN模型通过计算卡方统计量进行特征选择后准确率没有不进行特征选择时候高。 
3、但是GBDT模型在特征选择之后运行的速度比不进行特征选择快很多。    
4、可能原因是朴素贝叶斯和GBDT对特征选择鲁棒，而KNN因依赖高维空间距离度量敏感导致性能下降。此外，GBDT每棵树的分裂需要遍历所有特征，特征选择后特征维度降低，直接减少分裂时的计算量，所以运行时间减短。    
5、所以需要根据实际情况选择合适的处理方法以及模型进行分类。  

# 附：不进行特征选择

In [10]:
# 不进行特征选择
for name, model in models.items():
    # 处理均衡数据集
    model.fit(X_train_tfidf1, y_train1)
    y_pred_val1 = model.predict(X_val_tfidf1)
    print(f"\n{name}（类别均衡验证集）:")
    evaluate_model(y_val1, y_pred_val1)
    
    y_pred_test1 = model.predict(X_test_tfidf1)
    print(f"\n{name}（类别均衡测试集）:")
    evaluate_model(y_test1, y_pred_test1)
    print("\n" + "-"*60 + "\n")
    
    # 处理不均衡数据集
    model.fit(X_train_tfidf2, y_train2)
    y_pred_val2 = model.predict(X_val_tfidf2)
    print(f"\n{name}（类别不均衡验证集）:")
    evaluate_model(y_val2, y_pred_val2)
   
    y_pred_test2 = model.predict(X_test_tfidf2)
    print(f"\n{name}（类别不均衡测试集）:")
    evaluate_model(y_test2, y_pred_test2)
    print("\n" + "="*100 + "\n")


Naive Bayes（类别均衡验证集）:
Confusion Matrix:
预测类别   体育   家居   房产   教育   科技   财经
真实类别                              
体育    499    0    0    1    0    0
家居      1  453   27   11    6    2
房产      0    3  484    2    1   10
教育      1    3    1  479   14    2
科技      0    1    2    8  486    3
财经      0    0   13   15    6  466

Classification Report:
              precision    recall  f1-score   support

          体育       1.00      1.00      1.00       500
          家居       0.98      0.91      0.94       500
          房产       0.92      0.97      0.94       500
          教育       0.93      0.96      0.94       500
          科技       0.95      0.97      0.96       500
          财经       0.96      0.93      0.95       500

    accuracy                           0.96      3000
   macro avg       0.96      0.96      0.96      3000
weighted avg       0.96      0.96      0.96      3000

Macro F1 Score: 0.95564
Micro F1 Score: 0.95567

Naive Bayes（类别均衡测试集）:
Confusion Matrix:
预测类别   体育   家居   房产   教


GBDT（类别不均衡验证集）:
Confusion Matrix:
预测类别   体育   家居   房产   教育   科技   财经
真实类别                              
体育    484    1    0    1    1    0
家居      0  499    0    4    2    4
房产      0    2  508    0    0    3
教育      3   33    2  431   10    7
科技      2   27    2   13  479    6
财经      0   16    7    7    4  442

Classification Report:
              precision    recall  f1-score   support

          体育       0.99      0.99      0.99       487
          家居       0.86      0.98      0.92       509
          房产       0.98      0.99      0.98       513
          教育       0.95      0.89      0.92       486
          科技       0.97      0.91      0.93       529
          财经       0.96      0.93      0.94       476

    accuracy                           0.95      3000
   macro avg       0.95      0.95      0.95      3000
weighted avg       0.95      0.95      0.95      3000

Macro F1 Score: 0.94776
Micro F1 Score: 0.94767

GBDT（类别不均衡测试集）:
Confusion Matrix:
预测类别   体育   家居   房产   教育   科技   财经
